# Get SCE info from Suite2p Outputs - For Carmen

Here you just need to identify the suite2p output folder.  
Make sure the slashes are /, not backslashes.  
Confirm image sampling rate.

In [ ]:
suite2p_folder = "C:/Users/huynh/temp_local/carmen_january/suite2p_240109_240117_Plane0/"
imaging_sampling_rate = 10.31476027 # For carmen (2 planes on Bruker), basically same as Caro's (10.31475985). taken from XML

Get cell traces that are identified as cells by suite2p  
Get metadata

In [ ]:
import os

F = np.load(os.path.join(suite2p_folder, 'F.npy'))
iscell = np.load(os.path.join(suite2p_folder, 'iscell.npy'))

print('n_cells before suite2p filter (from iscell): ', F.shape[1])

traces = F[iscell[:,0]==1] # Confirmed in Cicada
n_cells, n_frames = traces.shape
print("n_cells chosen by suite2p: ", n_cells)
print("n_frames: ", n_frames)
print("seconds: ", n_frames/imaging_sampling_rate)
print("minutes: ", n_frames/imaging_sampling_rate/60)

Define some preprocessing functions and apply to traces    
This is based on JC's preprocessing for assembly detection 

In [ ]:
# Import libraries
import numpy as np
import matplotlib as plt
from scipy.stats import iqr
from scipy.signal import find_peaks

# Filters from JC's Matlab version

def median_normalization(traces):
    n_cells, n_frames = traces.shape
    for i in range(n_cells):
        traces[i, :] = traces[i, :] / np.median(traces[i, :])
    return traces

def bleaching_correction(traces):
    n_cells, n_frames = traces.shape
    for k in range(n_cells):
        p0 = np.polyfit(np.arange(n_frames), traces[k, :], 3)
        traces[k, :] = (traces[k, :] / np.polyval(p0, np.arange(n_frames)))-np.mean(traces[k,:]) # PHAN: replaced this poly with detrend to better match JC's Matlab version.
    return traces

def savitzky_golay_filt(traces):
    traces = signal.savgol_filter(traces, 7, 3, axis=1)  #PHAN:  changed framelength from 5 to 7 to match JC's Matlab version.
    return traces

In [ ]:
# Filters from JC's Matlab version
traces_for_sce = median_normalization(traces)
traces_for_sce = bleaching_correction(traces_for_sce)
traces_for_sce = savitzky_golay_filt(traces_for_sce)

## Detect when cells are active
Definition of active taken from JC.     
This step takes some time to process.  
To save time, I've exported the output (saved in same suite2p folder) to be retrieved later.   
To comment this box out (so it doesn't execute), highlight all the text (ctrl + a) and then ctrl + /.

In [ ]:
# # Detect SCE traces
n_cells, n_frames = traces_for_sce.shape

window_size = int(4 * imaging_sampling_rate)   # 4 seconds long window as Arnaud was doing with 40 in 10Hz recording
print(f"Window size is {window_size} frames")

minithreshold = 0.2
MinPeakDistance = 3

WinActive = [] # WinActive = np.where(speed > 1)[0]

th_cic = []
activity_tmp_all_cells = [[] for i in range(n_cells)]

for i in range(n_cells):
    activity_tmp_all_cells = [None] * n_cells
    for i in range(n_cells):
        th_i = max([3 * iqr(traces_for_sce[i]), 3 * np.std(traces_for_sce[i]), minithreshold])
        th_cic.append(th_i)
        peaks, properties = find_peaks(traces_for_sce[i], prominence=th_i, distance=MinPeakDistance)
        valeurs_identiques = np.intersect1d(peaks, WinActive)
        locs_sans_ide = np.setdiff1d(peaks, valeurs_identiques)
        activity_tmp_all_cells[i] = locs_sans_ide

#convert list of arrays to array of arrays in order to save to npy.file
activity_tmp_all_cells_array = np.array(activity_tmp_all_cells, dtype=object)

np.save(os.path.join(suite2p_folder, 'activity_tmp_all_cells.npy'), activity_tmp_all_cells_array)

If output is already saved, here you can retrieve.

In [ ]:
activity_tmp_all_cells_array = np.load(os.path.join(suite2p_folder, 'activity_tmp_all_cells.npy'), allow_pickle=True)
activity_tmp_all_cells = list(activity_tmp_all_cells_array)

Here we find timepoints where there are multiple active cells (over 10).  
This cell's output is the main info you're looking for.  The rest are just plots.

In [ ]:
from scipy import signal, stats

n_synchronous_frames = 2
sce_n_cells_threshold = 10 # 5
sce_min_distance = 4 # default from CICADA

raster = np.zeros((n_cells, n_frames))

for i in range(n_cells):
    raster[i, activity_tmp_all_cells[i]] = 1

# sum activity over n consecutive frames
sum_activity = np.zeros(n_frames - n_synchronous_frames)
for i in range(n_frames - n_synchronous_frames):
    sum_activity[i] = np.sum(np.amax(raster[:, np.arange(i, i + n_synchronous_frames)], axis=1))

# select synchronous calcium events
sce_loc = signal.find_peaks(sum_activity, height=sce_n_cells_threshold, distance=sce_min_distance)[0]
n_sce = len(sce_loc)

print("SCEs detected: ", n_sce)
print("SECs per second: ", n_sce/(n_frames/imaging_sampling_rate))
print("SECs per minute: ", n_sce/(n_frames/imaging_sampling_rate/60))

# create cells vs sce matrix
sce_cells_matrix = np.zeros((n_cells, n_sce))
for i in range(n_sce):
    sce_cells_matrix[:, i] = np.amax(raster[:, np.arange(np.max((sce_loc[i]-1, 0)),
                                                            np.min((sce_loc[i]+2, n_frames)))], axis=1)


Before plotting, I use TSNE to reorder the cells.  
(I don't think it helps much)

In [ ]:
from sklearn.manifold import TSNE

#### APPLY TSNE to reorder sce matrix

# Assuming Tr1b is your data, replace it with your actual data
Tr1b = sce_cells_matrix.copy()

# Set TSNE parameters
tsne = TSNE(
    n_components=1, # how many dimensions to reduce to, 1 is just a list, 2 for 2D, 3 for 3D, etc.
    n_iter=500,
    perplexity=1, #
    early_exaggeration=500,
    metric="correlation",
    method="barnes_hut",
    random_state=42  # Set a random state for reproducibility
)

# Fit TSNE and get the transformed data
TSNE_result = tsne.fit_transform(Tr1b)

# Get sorted indices
index = np.argsort(TSNE_result[:, 0])

# Sort the TSNE result based on the sorted indices
B = TSNE_result[index]

# Now B and index contain the equivalent results to the MATLAB code


Make raster plot (with and without TSNE cell ordering).  
This plot shows just the timepoints where there are at least ten cells active.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure and set the size
plt.figure(figsize=(20, 10))

# Define the number of rows and columns for the subplots
rows = 2
cols = 1

# Plot the first heatmap (top one)
plt.subplot(rows, cols, 1)
sns.heatmap(sce_cells_matrix, cmap='gray', cbar=False)

# Add title and axis labels for the top heatmap
plt.title('SCE cells matrix (unsorted)')
plt.xlabel('SCE (frame)')
plt.ylabel('Cell ID')

# Plot the second heatmap (bottom one)
plt.subplot(rows, cols, 2)
sns.heatmap(sce_cells_matrix[index], cmap='gray', cbar=False)

# Add title and axis labels for the bottom heatmap
plt.title('SCE cells matrix (sorted)')
plt.xlabel('SCE (frame)')
plt.ylabel('Cell ID')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


Lastly, a heatmap of all cell activities across all timepoints, but with SCE markers at the bottom.  

In [ ]:
# Try different normalization methods to make rasterplot more visible
#z-score traces
from scipy import stats
traces_zscore = stats.zscore(traces, axis=1)
zmin = np.percentile(traces_zscore, 1)  # Adjust percentile as needed
zmax = np.percentile(traces_zscore, 99)  # Adjust percentile as needed
print(zmin, '', zmax)

# min max normalize traces at each row
traces_min = np.min(traces, axis=1)
traces_max = np.max(traces, axis=1)
traces_minmax = (traces - traces_min[:, np.newaxis]) / (traces_max - traces_min)[:, np.newaxis]

# #plot histogram of traces_zscore
# plt.figure(figsize=(20,10))
# plt.hist(traces_zscore.flatten(), bins=100)
# plt.show()

To save the interactive heatmap in your suite2p folder, set "save_to_html" = True.

In [ ]:
import plotly.graph_objects as go
import numpy as np

save_to_html = False # check folder path
plot_traces = traces_zscore # traces_minmax # traces_zscore
order = index # isort # tsne_index

# plot trace average.
traces_sum = np.sum(traces, axis=0)
traces_avg = traces_sum / n_cells
traces_line = traces_avg # for plotting
traces_line = (traces_line - np.min(traces_line))*2
# Assuming traces_minmax, index, movements, imaging_sampling_rate, traces, and sce_loc are your data arrays

# Create a heatmap
heatmap_trace = go.Heatmap(
    z=plot_traces[order],
    colorscale='gray',
    showscale=False,
    name='Heatmap',  # Name for the legend
    zmin=zmin,  # Set a suitable minimum value for the colormap
    zmax=zmax,   # Set a suitable maximum value for the colormap
)

# Mark SCEs
sce_marker = go.Scatter(
    x=sce_loc,
    y=[0] * len(sce_loc),
    mode='markers',
    marker=dict(color='red', symbol='triangle-up', size=10),
    name='SCEs',  # Name for the legend
)

# Create layout with aspect ratio adjustment
layout = go.Layout(
    title='Cell activity with SCE markers',
    xaxis=dict(title='Time (frame)'),
    yaxis=dict(title='Cell ID'),
    height=600,  # Set the height of the plot
    width=1100,  # Set the width of the plot
)

# Create figure with adjusted legend order
fig = go.Figure(data=[heatmap_trace, sce_marker], layout=layout)

# Show the plot
fig.show()

if save_to_html:
    fig.write_html(os.path.join(suite2p_folder, "heatmap_sce.html"))